https://neo4j.com/docs/graph-data-science/current/getting-started/fastrp-knn-example/

In [16]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [17]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

In [18]:
neo4j_driver.execute_query(query_="""
CREATE
  (dan:Person {name: 'Dan'}),
  (annie:Person {name: 'Annie'}),
  (matt:Person {name: 'Matt'}),
  (jeff:Person {name: 'Jeff'}),
  (brie:Person {name: 'Brie'}),
  (elsa:Person {name: 'Elsa'}),

  (cookies:Product {name: 'Cookies'}),
  (tomatoes:Product {name: 'Tomatoes'}),
  (cucumber:Product {name: 'Cucumber'}),
  (celery:Product {name: 'Celery'}),
  (kale:Product {name: 'Kale'}),
  (milk:Product {name: 'Milk'}),
  (chocolate:Product {name: 'Chocolate'}),

  (dan)-[:BUYS {amount: 1.2}]->(cookies),
  (dan)-[:BUYS {amount: 3.2}]->(milk),
  (dan)-[:BUYS {amount: 2.2}]->(chocolate),

  (annie)-[:BUYS {amount: 1.2}]->(cucumber),
  (annie)-[:BUYS {amount: 3.2}]->(milk),
  (annie)-[:BUYS {amount: 3.2}]->(tomatoes),

  (matt)-[:BUYS {amount: 3}]->(tomatoes),
  (matt)-[:BUYS {amount: 2}]->(kale),
  (matt)-[:BUYS {amount: 1}]->(cucumber),

  (jeff)-[:BUYS {amount: 3}]->(cookies),
  (jeff)-[:BUYS {amount: 2}]->(milk),

  (brie)-[:BUYS {amount: 1}]->(tomatoes),
  (brie)-[:BUYS {amount: 2}]->(milk),
  (brie)-[:BUYS {amount: 2}]->(kale),
  (brie)-[:BUYS {amount: 3}]->(cucumber),
  (brie)-[:BUYS {amount: 0.3}]->(celery),

  (elsa)-[:BUYS {amount: 3}]->(chocolate),
  (elsa)-[:BUYS {amount: 3}]->(milk)
""")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x174575fd0>, keys=[])

In [19]:
neo4j_driver.execute_query(query_="""
MATCH (source:Person)-[r:BUYS]->(target:Product)
RETURN gds.graph.project(
  'purchases',
  source,
  target,
  {
    sourceNodeLabels: labels(source),
    targetNodeLabels: labels(target),
    relationshipType: 'BUYS',
    relationshipProperties: r { .amount }
  },
  { undirectedRelationshipTypes: ['BUYS'] }
)
""")

EagerResult(records=[<Record gds.graph.project(
  'purchases',
  source,
  target,
  {
    sourceNodeLabels: labels(source),
    targetNodeLabels: labels(target),
    relationshipType: 'BUYS',
    relationshipProperties: r { .amount }
  },
  { undirectedRelationshipTypes: ['BUYS'] }
)={'relationshipCount': 36, 'graphName': 'purchases', 'query': "\nMATCH (source:Person)-[r:BUYS]->(target:Product)\nRETURN gds.graph.project(\n  'purchases',\n  source,\n  target,\n  {\n    sourceNodeLabels: labels(source),\n    targetNodeLabels: labels(target),\n    relationshipType: 'BUYS',\n    relationshipProperties: r { .amount }\n  },\n  { undirectedRelationshipTypes: ['BUYS'] }\n)\n", 'projectMillis': 1337, 'configuration': {'readConcurrency': 4, 'undirectedRelationshipTypes': ['BUYS'], 'jobId': '8f310ce5-565d-48d0-995e-f0b0e4614000', 'logProgress': True, 'query': "\nMATCH (source:Person)-[r:BUYS]->(target:Product)\nRETURN gds.graph.project(\n  'purchases',\n  source,\n  target,\n  {\n    sourceNodeL

In [20]:
neo4j_driver.execute_query(query_="""
CALL gds.fastRP.mutate(
  'purchases',
  {
    embeddingDimension: 4,
    iterationWeights: [0.8, 1, 1, 1],
    relationshipWeightProperty: 'amount',
    randomSeed: 42,
    mutateProperty: 'embedding'
  }
)
YIELD nodePropertiesWritten
""")

EagerResult(records=[<Record nodePropertiesWritten=13>], summary=<neo4j._work.summary.ResultSummary object at 0x17453d610>, keys=['nodePropertiesWritten'])

In [21]:
neo4j_driver.execute_query(query_="""
CALL gds.knn.write(
  'purchases',
  {
    nodeProperties: ['embedding'],
    nodeLabels: ['Person', 'Product'],
    topK: 2,
    sampleRate: 1.0,
    deltaThreshold: 0.0,
    randomSeed: 42,
    concurrency: 1,
    writeProperty: 'score',
    writeRelationshipType: 'SIMILAR'
  }
)
YIELD similarityDistribution
RETURN similarityDistribution.mean AS meanSimilarity
""")

EagerResult(records=[<Record meanSimilarity=0.929229002732497>], summary=<neo4j._work.summary.ResultSummary object at 0x17458f8d0>, keys=['meanSimilarity'])

In [24]:
records, summary, keys = neo4j_driver.execute_query(query_="""
MATCH (n:Person)-[r:SIMILAR]->(m:Product)
RETURN n.name AS person1, m.name AS person2, r.score AS similarity
ORDER BY similarity DESCENDING, person1, person2
""")

print(records)

[<Record person1='Brie' person2='Cucumber' similarity=0.9944208860397339>, <Record person1='Matt' person2='Tomatoes' similarity=0.9895073175430298>, <Record person1='Brie' person2='Kale' similarity=0.9847311973571777>, <Record person1='Annie' person2='Tomatoes' similarity=0.9797690510749817>, <Record person1='Elsa' person2='Chocolate' similarity=0.9629020690917969>, <Record person1='Dan' person2='Chocolate' similarity=0.9248852729797363>, <Record person1='Jeff' person2='Tomatoes' similarity=0.8191928863525391>]
